## Linking viruses to hosts
- Predicting viral sequences using Genomad
- Using Minced to predict spacer regions (minced.smk)
- Need to use DRAM for functional annotation, and cas-genes

In [ ]:
## Running minced
# srun bc needs mem and time
srun --account=ctbrowngrp -p high2 -J minced -t 1:00:00 -c 10 --mem=50gb --pty bash
mamba activate minced

snakemake -s minced.smk --use-conda --resources mem_mb=50000 --rerun-triggers mtime \
-c 10 --rerun-incomplete -k -n

In [ ]:
# try cctyper:
cctyper genome.fa output_dir -t {threads}

### CRISPR sequences of ETEC
- Check if there are crispr-spacers in the etecs that are public
- Download sequences (needs: accession,name)
- Run minced
- blast x votus

In [ ]:
# download genomes
srun --account=ctbrowngrp -p med2 -J gbsketch -t 1:00:00 -c 2 --mem=10gb --pty bash

mamba activate branchwater-skipmer
sourmash scripts gbsketch -c 2 \
--keep-fasta --genomes-only etec_genbank.csv \
-f etec_genomes --download-only --failed etec.fail.csv

In [ ]:
# Run cctyper 
# download genomes
srun --account=ctbrowngrp -p med2 -J cctyper -t 4:00:00 -c 12 --mem=20gb --pty bash

snakemake -s cctyper.smk --use-conda --resources mem_mb=20000 --rerun-triggers mtime \
-c 12 --rerun-incomplete -k -n

In [ ]:
rule directsketch:
    input:
        csv = f"{OUTPUT_DIR}/{pang_name_out}/{pang_name_out}xgtdb.csv",
    output:
        sig = f"{OUTPUT_DIR}/{pang_name_out}/sourmash/{pang_name_out}.gtdb.zip",
        failed_test = f"{OUTPUT_DIR}/{pang_name_out}/check/{pang_name_out}.failed.csv",
        fail_checksum= f"{OUTPUT_DIR}/{pang_name_out}/check/{pang_name_out}.checksum.failed.csv",
    conda: 
        "branchwater-skipmer"
    threads: 10
    params:
        output_folder=f"{OUTPUT_DIR}/{pang_name_out}/MAGs"
    shell:
        """ 
        sourmash scripts gbsketch  --keep-fasta --genomes-only \
        {input.csv} -o {output.sig} -p dna,k=21,k=31,scaled=100,abund \
        -f {params.output_folder} -k -c {threads} \
        --failed {output.failed_test} -r 1 --checksum-fail {output.fail_checksum}

In [ ]:
# Making a blastdb of all the crispr sequences, or all the vir seqs?